In [37]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


In [38]:
api_key = "78d406f975874c20ac2298c6d83e2f7d"
azure_endpoint = "https://nitoropenai.openai.azure.com/"
api_version = "2022-12-01"

llm = AzureOpenAI(
    model="text-davinci-003",
    deployment_name="text-davinci-003",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [39]:
# You need to deploy your own embedding model as well as your own chat completion model

# using azure-openai-embeddings
# embed_model = AzureOpenAIEmbedding(
#     model="text-embedding-ada-002",
#     deployment_name="text-embedding-ada-002",
#     api_key=api_key,
#     azure_endpoint=azure_endpoint,
#     api_version=api_version,
# )

# using hugging-face-embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [40]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
! pip install llama-index-readers-file


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [41]:
from llama_index.readers.file import PyMuPDFReader

reader = PyMuPDFReader()
documents = reader.load_data("./docs/finance.pdf")

In [45]:
# using chromadb vector storage
db = chromadb.PersistentClient(path="./chroma_db_1")
chroma_collection = db.get_or_create_collection("chroma_coll_finance")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


In [23]:
# using default vector storage

# index = VectorStoreIndex.from_documents(documents)
# index.storage_context.persist(persist_dir="./qna_storage")

In [46]:
documents

[Document(id_='80b1bce8-ba7c-423f-8feb-2651bef2b723', embedding=None, metadata={'total_pages': 37, 'file_path': './docs/finance.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Revolutionizing Finance with LLMs: An\nOverview of Applications and Insights\nHuaqin Zhao, Zhengliang Liu, Zihao Wu, Yiwei Li, Tianze Yang,\nPeng Shu, Shaochen Xu, Haixing Dai, Lin Zhao, Gengchen Mai,\nNinghao Liu, Tianming Liu†∗†\nJanuary 23, 2024\nAbstract\nIn recent years, Large Language Models (LLMs) like ChatGPT have\nseen considerable advancements and have been applied in diverse fields.\nBuilt on the Transformer architecture, these models are trained on exten-\nsive datasets, enabling them to understand and generate human language\neffectively. In the financial domain, the deployment of LLMs is gaining\nmomentum. These models are being utilized for automating financial re-\nport generation, forecasting market trends, analyzing investor sentiment

In [47]:
query_engine = index.as_query_engine(
                                    similarity_top_k=5
                                )

In [59]:
response = query_engine.query("What are the different usecases where LLMs can help in finance")

In [60]:
response

Response(response=' LLMs can be used in finance for tasks such as automating financial report generation, forecasting market trends, analyzing investor sentiment, and offering personalized financial advice. LLMs can also be used to distill key insights from vast financial data, aiding institutions in making informed investment choices and enhancing both operational efficiency and customer satisfaction.', source_nodes=[NodeWithScore(node=TextNode(id_='8aa0b97b-5c9c-466e-9322-e889658b3401', embedding=None, metadata={'total_pages': 37, 'file_path': './docs/finance.pdf', 'source': '3'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c1bdd320-34c9-4590-9f9a-bc943fc1cc74', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'total_pages': 37, 'file_path': './docs/finance.pdf', 'source': '3'}, hash='039d12776a5427a97014f6e3f03a36a65f20b55cc24cfd3b175ac0fef6024ce2'), <NodeRelationship.PREVIOUS: '2'>: RelatedNo

In [61]:
response.response

' LLMs can be used in finance for tasks such as automating financial report generation, forecasting market trends, analyzing investor sentiment, and offering personalized financial advice. LLMs can also be used to distill key insights from vast financial data, aiding institutions in making informed investment choices and enhancing both operational efficiency and customer satisfaction.'

In [62]:
response.get_formatted_sources()

'> Source (Doc id: 8aa0b97b-5c9c-466e-9322-e889658b3401): reliable [42].\nTo address these issues, researchers and developers are continuously refining\nthe ...\n\n> Source (Doc id: eb69b781-91b4-4949-b33c-bb5ef6e72c31): Figure 1: An overview of the LLMs’ capacities in financial tasks.\n1\nIntroduction\nOver the past fe...\n\n> Source (Doc id: 374d4264-77f4-4126-a53f-026884d6e115): Revolutionizing Finance with LLMs: An\nOverview of Applications and Insights\nHuaqin Zhao, Zhenglia...\n\n> Source (Doc id: fa7121a9-5a26-48f6-9c1f-78e38befa290): straints of LLMs in the financial domain.\nCentral to our findings is the re-\nmarkable adeptness o...\n\n> Source (Doc id: b7367eb3-48ae-46a7-a230-f0fd7264fe50): Figure 2: The Ability of LLMs in Financial Tasks\nAttention(q, k, v) = softmax( qk\n√dk\n)v\n(1)\nSelf...'